In [1]:
using DataFrames
using DataFramesMeta
using LinearAlgebra
using CSV 
using BenchmarkTools
using NLopt
using Distributions
using StatsFuns
using Primes
using Statistics
using Random

In [2]:
incomeMeans = [2.01156, 2.06526, 2.07843, 2.05775, 2.02915, 2.05346, 2.06745,
2.09805, 2.10404, 2.07208, 2.06019, 2.06561, 2.07672, 2.10437, 2.12608, 2.16426,
2.18071, 2.18856, 2.21250, 2.18377]
 
sigma_v = 1.72
βs = [-7.061, 2.883, 1.521, -0.122, 3.46]
γs = [0.952, 0.477, 0.619, -0.415, -0.046, 0.019]
α = 43.501
σs = [3.612, 4.628, 1.818, 1.050, 2.056];

## Data Preparation

In [3]:
cars = DataFrame(CSV.File("/Users/wujifan/Desktop/coding/blp/blp_1995_data.csv", header = true));
 
cars[!,:ln_hpwt] = log.(cars[!,:hpwt])
cars[!,:ln_space] = log.(cars[!,:space])
cars[!,:ln_mpg] = log.(cars[!,:mpg])
cars[!,:ln_mpd] = log.(cars[!,:mpd])
cars[!,:ln_price] = log.(cars[!,:price])
cars[!,:trend] = cars[!,:market] .- 1 
cars.cons = ones(size(cars,1))

cars.diff_2 = log.(cars.share) - log.(cars.share_out);

In [4]:
markets = cars.market
m = unique(markets)
firmid = cars.firmid;

In [5]:
X_d = Matrix(cars[:,[:cons,:hpwt,:air,:mpd,:space]])
X_s = Matrix(cars[:,[:cons,:ln_hpwt,:air,:ln_mpg,:ln_space,:trend]])
price = cars.price
δ = convert(Array, round.(cars[:,:diff_2], digits = 20));
share = cars.share;

In [6]:
function gen_iv(X, markets, firmid, unique_markets)
    z_1 = similar(X)
    z_2 = similar(X)
    for m in unique_markets
        X_sub = X[findall(markets .== m),:]
        firmid_sub = firmid[findall(markets .== m),:]
        samefirm = convert(Array{Float64,2}, firmid_sub .== firmid_sub')
        z1 = similar(X_sub)
        z2 = similar(X_sub)
        for i in 1:size(X,2)
            X_i = X_sub[:,i]
            #z_1[:,i] = samefirm * X_i
            Xi_sum = sum(X_i,dims = 1)
            z1[:,i] = sum((X_i .* samefirm), dims = 2)'
            z2[:,i] .= Xi_sum 
        end
        z_1[findall(markets .== m),:] = z1
        z_2[findall(markets .== m),:] = z2
    end
    return z_1,z_2
end

gen_iv (generic function with 1 method)

## Table III Column(1)

In [7]:
X = [X_d price]
Y = δ
β = inv(X'*X)*X'*Y


6-element Vector{Float64}:
 -10.073007506006823
  -0.1230948774685233
  -0.03441478166781374
   0.2654656794440304
   2.3419141640983057
  -0.08860627178125562

## Table III Column(2)

In [8]:
zd_1, zd_2 = gen_iv(X_d,markets,firmid,m)
Z_d = [X_d zd_1 zd_2]
X = [X_d price]
Y = δ
β₁ = inv(X'*Z_d*Z_d'*X)*X'*Z_d*Z_d'*Y
e = Y - X*β₁
mm = Z_d .* e
g = mean(mm, dims=1)
w = inv(mm'mm/size(mm,1) .- g.*g')
β₂ = inv(X'*Z_d*w*Z_d'*X)*X'*Z_d*w*Z_d'*Y


6-element Vector{Float64}:
 -9.2762929388534
  1.9493511517597137
  1.2873914769799415
  0.054561470890819044
  2.357602541609871
 -0.2157869518896273

## Table III Column(3)

In [9]:
β = inv(X_s' * X_s) *X_s' * log.(price) 

6-element Vector{Float64}:
  2.7929042302809473
  0.5203366842873054
  0.6797512956744369
 -0.4706402745568128
  0.12482708487573904
  0.01283074614345114

# Full Model

In [10]:
function cal_share(X,price,delta,unique_markets,alpha,sigma,sim)
    Random.seed!(2024)
    p_jt = zeros(size(X,1),1)
    p_ijt = zeros(size(X,1),sim)'
    v_im = randn(sim,5)
    y_v = randn(sim,1)
    for m in unique_markets
        y_im = exp.(incomeMeans[m] .+ sigma_v .*y_v)
        X_sub = X[findall(markets .== m),:]
        price_sub = price[findall(markets .== m),:]
        rand_effect = -alpha.* (1 ./ y_im) * price_sub' + v_im.*sigma'*X_sub'
        delta_sub = delta[findall(markets .== m),:]
        delta_ma = repeat(delta_sub',inner=[sim,1])
        s0 = 1 ./ (1 .+ sum(exp.(rand_effect + delta_ma),dims = 2))
        p_ijt_m = exp.(rand_effect + delta_ma) .* s0
        p_jt[findall(markets .== m),:] = mean(p_ijt_m,dims = 1)
        p_ijt[:,findall(markets .== m)] = p_ijt_m
    end
    return p_ijt',p_jt
end

cal_share (generic function with 1 method)

In [11]:
function solve_delta(X,price,delta,s,unique_markets,alpha,sigma,sim,eps,tol)
    delta_0 = delta
    while eps > tol
        p_ijt,p_jt = cal_share(X,price,delta_0,unique_markets,alpha,sigma,sim)
        delta_1 = delta_0 + log.(s./p_jt)
        eps = maximum(abs.((delta_1 - delta_0)))
        delta_0 = delta_1
    end
    return delta_0
end

solve_delta (generic function with 1 method)

In [12]:
function cal_mc(X,price,delta,s,unique_markets,alpha,sigma,sim)
    mc = zeros(size(X,1),1)
    Random.seed!(2024)
    y_v = randn(sim,1)
    p_ijt,p_jt = cal_share(X,price,delta,unique_markets,alpha,sigma,sim)
    for m in unique_markets
        firmid_sub = firmid[findall(markets .== m),:]
        Ω_sum = zeros(size(firmid_sub,1),size(firmid_sub,1))
        samefirm = convert(Array{Float64,2}, firmid_sub .== firmid_sub')
        s_m = p_jt[findall(markets .== m),:]
        p_m = price[findall(markets .== m),:]
        for i in 1:sim
            s_i = p_ijt[findall(markets .== m),i]
            y_i = exp.(incomeMeans[m] .+ sigma_v .*y_v)[i]
            Ω_sum += (alpha ./ y_i) .* samefirm * (s_i * s_i' - diagm(s_i))
        end
        Ω =  Ω_sum ./ sim
        mc[findall(markets .== m),:] = p_m - pinv(Ω) * s_m
    end
    return mc
end

cal_mc (generic function with 1 method)

In [13]:
mutable struct Model_Data
    X_d :: Matrix{Float64}
    X_s :: Matrix{Float64}
    price :: Vector{Float64}
    delta :: Vector{Float64}
    share :: Vector{Float64}
    unique_markets :: Vector{Int64}
    alpha :: Float64
    sigma :: Vector{Float64}
    sim :: Int64
    eps :: Int64
    tol :: Float64
    firmid :: Vector{Int64}
    markets :: Vector{Int64}
    mc :: Vector{Float64}
    theta1 :: Vector{Float64}
    function Model_Data(X_d,X_s,price,delta,share,unique_markets,alpha,sigma,sim,eps,tol,firmid,markets,mc,theta1)
        return new(X_d,X_s,price,delta,share,unique_markets,alpha,sigma,sim,eps,tol,firmid,markets,mc,theta1)
    end
end

In [14]:
function update_y(m::Model_Data)
    X_d = m.X_d
    X_s = m.X_s
    price = m.price
    delta = m.delta
    s = m.share
    unique_markets = m.unique_markets
    alpha = m.alpha
    sigma = m.sigma
    sim = m.sim
    eps = m.eps
    tol = m.tol
    markets = m.markets
    m.delta = vec(solve_delta(X_d,price,delta,s,unique_markets,alpha,sigma,sim,eps,tol))
    delta = m.delta
    mc = vec(cal_mc(X_d,price,delta,s,unique_markets,alpha,sigma,sim))
    Y = vcat(delta,log.(mc))
    Z_d1,Z_d2 = gen_iv(X_d,markets,firmid,unique_markets)
    Z_d = [Z_d1 Z_d2]
    Z_s1,Z_s2 = gen_iv(X_s,markets,firmid,unique_markets)
    Z_s = [Z_s1 Z_s2]
    X = [X_d zeros(size(X_s)); zeros(size(X_d)) X_s]
    Z = [Z_d zeros(size(Z_s)); zeros(size(Z_d)) Z_s]
    return X,Y,Z
end

update_y (generic function with 1 method)

In [15]:
function gmm(θ₂,m::Model_Data,W)
    m.alpha = θ₂[6]
    m.sigma = θ₂[1:5] 
    X,Y,Z = update_y(m)
    θ₁ = inv(X'*Z*W*Z'*X)*X'*Z*W*Z'*Y
    m.theta1 = θ₁
    ξ = Y - X* θ₁
    g = Z'*ξ
    obj = g'* W * g
    return obj
end

gmm (generic function with 1 method)

In [16]:
model = Model_Data(X_d,X_s,price,δ,share,m,α,σs,1000,10,1e-14,firmid,markets,zeros(size(share,1)),vec(zeros(11,1)));

In [17]:
Z_d1,Z_d2 = gen_iv(X_d,markets,firmid,m)
Z_d = [Z_d1 Z_d2]
Z_s1,Z_s2 = gen_iv(X_s,markets,firmid,m)
Z_s = [Z_s1 Z_s2]
Z = [Z_d zeros(size(Z_s)); zeros(size(Z_d)) Z_s];

W = inv(Z'Z)
opt = Opt(:LN_NELDERMEAD, 6)  # 创建一个6维度优化问题的Opt对象

# 现在才可以设置opt对象的参数
lower_bounds!(opt, [0.0, 0.0, 0.0, 0.0, 0.0, 5.0])
initial_step!(opt, [0.5,0.5,0.5,0.5,0.5,3].*0.1)
xtol_rel!(opt, 1e-4)
maxeval!(opt, 10)
min_objective!(opt, (x, y) -> gmm(x, model,W))  # 确保这个函数签名正确


@time minf_, minx_, ret_ = optimize(opt,[3.612,4.628,1.818,1.050,2.056,43.501])


177.152172 seconds (14.99 M allocations: 462.981 GiB, 27.82% gc time, 0.80% compilation time)


(623.467086749299, [3.637, 4.652999999999999, 1.843, 0.9499999999999993, 2.0809999999999986, 43.65099999999999], :MAXEVAL_REACHED)

In [18]:
X,Y,Z = update_y(model)
θ₁ =  model.theta1
ξ = Y - X*θ₁
mm = Z .* ξ
g = mean(mm, dims=1)
W_opt = inv(mm'mm/size(mm,1) .- g.*g');

In [19]:
opt = Opt(:LN_NELDERMEAD, 6)  # 创建一个6维度优化问题的Opt对象

# 现在才可以设置opt对象的参数
lower_bounds!(opt, [0.0, 0.0, 0.0, 0.0, 0.0, 5.0])
initial_step!(opt, [0.5,0.5,0.5,0.5,0.5,3].*0.1)
xtol_rel!(opt, 1e-4)
maxeval!(opt, 10)
min_objective!(opt, (x, y) -> gmm(x, model,W_opt))  # 确保这个函数签名正确


@time minf_, minx_, ret_ = optimize(opt,[3.612,4.628,1.818,1.050,2.056,43.501])

140.796176 seconds (11.23 M allocations: 453.240 GiB, 17.06% gc time, 0.00% compilation time)


(2.101765484295921e6, [3.637, 4.652999999999999, 1.7930000000000004, 1.075, 1.9560000000000004, 43.65099999999999], :MAXEVAL_REACHED)